In [1]:
import pandas as pd
import os
import datetime as dt

thedir = os.getcwd()
writedir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/interim'))
extdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/external'))

In [2]:
# create archival data

maison_blanche_mean = [8.5,10.7,14.3,15.1,17.3,"nan","nan"]
vogelsang_mean = [10.3,12.2,16.1,"nan","nan",14.7,9.4]
boezingenmoos_mean = [10.2,12.1,16.1,16.5,17.9,14.5,9.2]
maison_min = [-1.2,0.6,6.5,6.2,6.2,3.1,'nan']
vogel_min = [0.7,2.5,3.7,8.6,'nan',4.4,0.9]
boz_min = [-1.9,0.6,6.5,6.2,6.2,3.1,-0.9]
maison_max = [18.9,22.3,25.5,24.4,26.4,20.3,18.4]
vogel_max = [21.8,24.3,27.9,27.7,29.6,22.4,20.5]
boz_max = [22.1,25.3,29.6,28.6,31.2,24.1,21.8,]

maison=pd.DataFrame([maison_min,maison_max,maison_blanche_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])
maison.columns = [x + '-' + '1983' for x in maison.columns]
maison = maison.transpose()
maison.columns = [x + '-' + 'maison' for x in maison.columns]

vogel=pd.DataFrame([vogel_min,vogel_max,vogelsang_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])
vogel.columns = [x + '-' + '1983' for x in vogel.columns]
vogel = vogel.transpose()
vogel.columns = [x + '-' + 'vogel' for x in vogel.columns]

boz=pd.DataFrame([boz_min,boz_max,boezingenmoos_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])
boz.columns = [x + '-' + '1983' for x in boz.columns]
boz = boz.transpose()
boz.columns  = [x + '-' + 'boz' for x in boz.columns]

archive = pd.concat([boz,maison,vogel],axis = 1)

archive.reset_index(inplace=True, drop = False)

In [3]:
#read in other data
data_lcd = pd.read_csv(F'{writedir}/alldata.csv')
data_os = pd.read_csv(F'{extdir}/opensense-temp.csv')
data_be = pd.read_csv(F'{extdir}/ref-2m-3m-be-2018.csv')

# correct date-time column
def fix_date_string(x):
    x = x[:10] + " " + x[11:-5]
    return x
data_os['time_int'] = data_os.createdAt.apply(lambda x: fix_date_string(x))
data_os['md'] = pd.to_datetime(data_os.time_int,format = '%Y/%m/%d %H:%M:%S')

data_lcd["md"] = pd.to_datetime(data_lcd.time,format = '%Y/%m/%d %H:%M:%S')
data_ = data_lcd[["md","Log_201"]].copy()

data_be['md'] = pd.to_datetime(data_be['Zeit'], infer_datetime_format=True)

archive['md'] = pd.to_datetime(archive['index'], infer_datetime_format=True)

In [15]:
class Sensors(object):
    def __init__(self, project=None, name=None, city=None, sensor_type=None, data = None, elev=None, coord=None, quant = None):
        self.project = project
        self.name = name
        self.city = city
        self.data = data
        self.sensor_type = sensor_type
        self.coord = coord
        self.elev = elev
        self.quant = quant
    def get_month(self):
        x = self.data
        x['month'] = x.md.dt.to_period("m")
        return x

class OpenSensors(Sensors):    
    def get_day(self):
        x = self.data
        x['day'] = x.md.dt.to_period("d")
        return x
    
class ArchiveSensors(Sensors):
    def get_monthly(self):
        pass

class LowCostSensors(Sensors):
    def get_day(self):
        x = self.data
        x['day'] = x.md.dt.to_period("d")
        return x
    def get_mean(self,time):
        x = self.data
        daily_mean = x.groupby([time]).agg(mean)
        return daily_mean
    def get_mean(self,time):
        x = self.data
        daily_max = x.groupby([time]).agg(max)
        return daily_max
    def get_mean(self,time):
        x = self.data
        daily_min = x.groupby([time]).agg(min)
        return daily_min
    def get_mean(self,time):
        x = self.data
        daily_var = x.groupby([time]).agg(var)
        return daily_var

In [16]:
# read in opensensor
os_1 = OpenSensors('OpenSense','Gumme Brügg','Brügg','custom',data_os,469,(47.126465,7.285249))

# read in Biel Sensors
elevation= {"Log_202":430, "Log_201":432,"Log_203":433,"Log_204":430,"Log_205":439,"Log_206":437,"Log_207":430}
coord= {"Log_202":(47.130669,7.236258), "Log_201":(47.130792,7.241046),"Log_203": (47.136637, 7.246960),"Log_204":(47.141086,7.253485),"Log_205":(47.144746,7.265149),"Log_206":(47.138338,7.295326),"Log_207":(47.179081,7.415102)}
city= {"Log_202":'Biel', "Log_201":'Biel',"Log_203":'Biel',"Log_204":'Biel',"Log_205":'Biel',"Log_206":'Orpund',"Log_207":'Grenchen'}

log_cols = [ x for x in data_lcd.columns if x[:3] == "Log" ]

mylist = []
for i in log_cols:
    data = data_lcd[['md',i]].copy()
    mylist.append(LowCostSensors('biel-temps',i,city[i],'lcd',data,elevation[i],coord[i]))
biel_sensors = mylist

# read in archive sensors
elevation = {'mai':720, 'vog':544, 'boz':441}
coord = {'mai':(47.145999,7.242621), 'vog': (47.12456,7.242723), 'boz': (47.15189,7.272195)}
city = {'mai':'evilard', 'vog':'biel', 'boz':'biel'}

lgm = [ x for x in archive.columns if "mai" in x]
lgv = [ x for x in archive.columns if "vog" in x]
lgb = [ x for x in archive.columns if "boz" in x]
lgm.append('md')
lgv.append('md')
lgb.append('md')

cols = [lgm,lgv,lgb]

mylist = []
for i in cols:
    data = archive[i].copy()
    if 'mai' in i[0]:
        key = 'mai'
        mylist.append(ArchiveSensors('archive',key,city[key],'lcd',data,elevation[key],coord[key]))
    elif 'boz' in i[0]:
        key = 'boz'
        mylist.append(ArchiveSensors('archive',key,city[key],'lcd',data,elevation[key],coord[key]))
    elif 'vog' in i[0]:
        key = 'vog'
        mylist.append(ArchiveSensors('archive',key,city[key],'lcd',data,elevation[key],coord[key]))
archival = mylist

In [18]:
# get month and day columns for aggregation
[x.get_month() for x in archival]
[x.get_month() for x in biel_sensors]
[x.get_day() for x in biel_sensors]
os_1.get_month()
os_1.get_day()

,lat,lon,height,boxName,boxId,exposure,unit,value,createdAt,sensorId,sensorType,time_int,md,month,day
0,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,20.5,2022-08-10T21:58:34.724Z,5c9e06c1cbf9ae001a48c976,SI,2022-08-10 21:58:34,2022-08-10 21:58:34,2022-08,2022-08-10
1,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,20.6,2022-08-10T21:53:22.346Z,5c9e06c1cbf9ae001a48c976,SI,2022-08-10 21:53:22,2022-08-10 21:53:22,2022-08,2022-08-10
2,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,20.8,2022-08-10T21:47:40.121Z,5c9e06c1cbf9ae001a48c976,SI,2022-08-10 21:47:40,2022-08-10 21:47:40,2022-08,2022-08-10
3,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,20.9,2022-08-10T21:41:56.775Z,5c9e06c1cbf9ae001a48c976,SI,2022-08-10 21:41:56,2022-08-10 21:41:56,2022-08,2022-08-10
4,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,20.7,2022-08-10T21:36:17.936Z,5c9e06c1cbf9ae001a48c976,SI,2022-08-10 21:36:17,2022-08-10 21:36:17,2022-08,2022-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5348,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,17.1,2022-07-08T05:43:32.130Z,5c9e06c1cbf9ae001a48c976,SI,2022-07-08 05:43:32,2022-07-08 05:43:32,2022-07,2022-07-08
5349,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,16.8,2022-07-08T05:26:38.710Z,5c9e06c1cbf9ae001a48c976,SI,2022-07-08 05:26:38,2022-07-08 05:26:38,2022-07,2022-07-08
5350,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,16.7,2022-07-08T05:20:42.742Z,5c9e06c1cbf9ae001a48c976,SI,2022-07-08 05:20:42,2022-07-08 05:20:42,2022-07,2022-07-08
5351,47.126465,7.285249,NaN,Gumme Brügg,5c9e06c1cbf9ae001a48c972,outdoor,°C,16.6,2022-07-08T05:15:11.158Z,5c9e06c1cbf9ae001a48c976,SI,2022-07-08 05:15:11,2022-07-08 05:15:11,2022-07,2022-07-08


,md,Log_206,month,day
0,2022-07-09 00:00:00,15.378,2022-07,2022-07-09
1,2022-07-09 00:10:00,14.517,2022-07,2022-07-09
2,2022-07-09 00:20:00,13.942,2022-07,2022-07-09
3,2022-07-09 00:30:00,13.750,2022-07,2022-07-09
4,2022-07-09 00:40:00,14.038,2022-07,2022-07-09
...,...,...,...,...
4808,2022-08-11 09:20:00,21.091,2022-08,2022-08-11
4809,2022-08-11 09:30:00,21.187,2022-08,2022-08-11
4810,2022-08-11 09:40:00,21.187,2022-08,2022-08-11
4811,2022-08-11 09:50:00,21.664,2022-08,2022-08-11


In [7]:
a.get_month()
b.get_day()

NameError: name 'a' is not defined

In [ ]:
c = biel_sensors[0]
c.data